In [22]:
import os

In [23]:
%pwd

'e:\\TextSummarizer'

In [3]:
os.chdir("..")

In [24]:
%pwd

'e:\\TextSummarizer'

In [25]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir:Path
    data_path: Path
    tokenizer_name:Path


In [29]:
from src.textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_tranformation_config(self)->DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_tranform_config=DataTransformationConfig(

            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_tranform_config



In [40]:
import os
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk
from src.textsummarizer.logging import logging


class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def words_into_tokens(self,dataset):
        input_encodings=self.tokenizer(dataset['dialogue'],max_length=1024,truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(dataset['summary'], max_length = 128, truncation = True )

        return{
                "input_ids" : input_encodings["input_ids"],
                'attention_mask': input_encodings['attention_mask'],
                'labels': target_encodings['input_ids']
            }

    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt=dataset_samsum.map(self.words_into_tokens, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
        

In [41]:
try:
    config=ConfigurationManager()
    data_tranform_config=config.get_data_tranformation_config()
    data_tranformation=DataTransformation(config=data_tranform_config)
    data_tranformation.convert()
except Exception as e:
    raise e

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]e:\TextSummarizer\venv\lib\site-packages\transformers\tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 12788.80 examples/s]
